In [1]:
from speechbrain.lobes.augment import TimeDomainSpecAugment
from speechbrain.lobes.augment import EnvCorrupt
from speechbrain.dataio.dataio import read_audio
import numpy as np
import random
import torch
import torchaudio
import pandas as pd

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.


In [2]:
#!pip install speechbrain

# Modification des pistes audio

In [3]:
def corrupter(audio_file,save_name):

    signal = read_audio(audio_file)
    clean = signal.unsqueeze(0)
    do_augment = TimeDomainSpecAugment(speeds=[i for i in range(85, 116)],
                                    perturb_prob=1, 
                                    drop_freq_prob=1, 
                                    drop_chunk_prob=1,
                                    drop_chunk_length_low=1000,
                                    drop_freq_count_low=0,
                                    drop_freq_count_high=np.random.randint(50, size=1)[0],
                                    drop_chunk_count_low=0,
                                    drop_chunk_count_high=np.random.randint(50, size=1)[0],
                                    drop_chunk_length_high=5000)
    do_augment_env = EnvCorrupt(reverb_prob=1.0,
                              noise_prob=1.0,
                              noise_num_workers=np.random.randint(100, size=1)[0]
                              )
    augmented_signal = do_augment(clean, torch.ones(1))
    augmented_signal_env = do_augment_env(augmented_signal, torch.ones(1))
    torchaudio.save(save_name, augmented_signal_env, sample_rate=16000)

In [4]:
tab=pd.read_csv('SPEAKERS_TYPE.csv',';')
tab.groupby('ORIGIN').count()

C:\Users\Paul\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,ID,Name,SEX
ORIGIN,,,
TEST,50,50,50
TRAIN,201,201,201


In [5]:

tab2=tab[tab['ORIGIN']=='TRAIN']

In [6]:
tab2

,ID,Name,ORIGIN,SEX
0,19,LibriSpeech/19-198-0000.flac,TRAIN,F
1,26,LibriSpeech/26-495-0000.flac,TRAIN,M
2,27,LibriSpeech/27-123349-0000.flac,TRAIN,M
3,32,LibriSpeech/32-21625-0000.flac,TRAIN,F
4,40,LibriSpeech/40-121026-0000.flac,TRAIN,F
...,...,...,...,...
196,8747,LibriSpeech/8747-293952-0000.flac,TRAIN,M
197,8770,LibriSpeech/8770-295462-0000.flac,TRAIN,M
198,8797,LibriSpeech/8797-294123-0000.flac,TRAIN,M
199,8838,LibriSpeech/8838-298545-0000.flac,TRAIN,M


In [7]:
tab2.sort_values(by='ID')

,ID,Name,ORIGIN,SEX
0,19,LibriSpeech/19-198-0000.flac,TRAIN,F
1,26,LibriSpeech/26-495-0000.flac,TRAIN,M
2,27,LibriSpeech/27-123349-0000.flac,TRAIN,M
3,32,LibriSpeech/32-21625-0000.flac,TRAIN,F
4,40,LibriSpeech/40-121026-0000.flac,TRAIN,F
...,...,...,...,...
196,8747,LibriSpeech/8747-293952-0000.flac,TRAIN,M
197,8770,LibriSpeech/8770-295462-0000.flac,TRAIN,M
198,8797,LibriSpeech/8797-294123-0000.flac,TRAIN,M
199,8838,LibriSpeech/8838-298545-0000.flac,TRAIN,M


In [8]:
n0=tab['ID'].max()+1
for i in range(len(tab2['Name'])):
    save_n='LibriSpeech/'+str(n0+i)+'.flac'
    corrupter(tab2['Name'].iloc[i],save_n)
    L=pd.DataFrame([[n0+i,save_n,'TRAIN',tab2['SEX'].iloc[i]]],columns=['ID','Name','ORIGIN','SEX'])
    tab2=tab2.append(L,ignore_index=True)
    

In [9]:
tab2

,ID,Name,ORIGIN,SEX
0,19,LibriSpeech/19-198-0000.flac,TRAIN,F
1,26,LibriSpeech/26-495-0000.flac,TRAIN,M
2,27,LibriSpeech/27-123349-0000.flac,TRAIN,M
3,32,LibriSpeech/32-21625-0000.flac,TRAIN,F
4,40,LibriSpeech/40-121026-0000.flac,TRAIN,F
...,...,...,...,...
397,9172,LibriSpeech/9172.flac,TRAIN,M
398,9173,LibriSpeech/9173.flac,TRAIN,M
399,9174,LibriSpeech/9174.flac,TRAIN,M
400,9175,LibriSpeech/9175.flac,TRAIN,M


In [10]:
tab=tab[tab['ORIGIN']=='TEST'].append(tab2)

In [11]:
tab.to_csv('SPEAKERS_TYPE_extended.csv',';',index=False)

## Inversion des pistes audios pour rajouter des audios

In [12]:
tab=pd.read_csv('SPEAKERS_TYPE_extended.csv',';')
tab2=tab[tab['ORIGIN']=='TRAIN']
tab2

C:\Users\Paul\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,ID,Name,ORIGIN,SEX
50,19,LibriSpeech/19-198-0000.flac,TRAIN,F
51,26,LibriSpeech/26-495-0000.flac,TRAIN,M
52,27,LibriSpeech/27-123349-0000.flac,TRAIN,M
53,32,LibriSpeech/32-21625-0000.flac,TRAIN,F
54,40,LibriSpeech/40-121026-0000.flac,TRAIN,F
...,...,...,...,...
447,9172,LibriSpeech/9172.flac,TRAIN,M
448,9173,LibriSpeech/9173.flac,TRAIN,M
449,9174,LibriSpeech/9174.flac,TRAIN,M
450,9175,LibriSpeech/9175.flac,TRAIN,M


In [13]:
def reverser(audio_file,save_name):
    audio_normal = read_audio(audio_file)
    clean = audio_normal.unsqueeze(0)
    rTensor = torch.fliplr(clean)
    torchaudio.save(save_name, rTensor, sample_rate=16000)

In [14]:
n0=tab['ID'].max()+1
for i in range(len(tab2['Name'])):
    save_n='LibriSpeech/'+str(n0+i)+'.flac'
    reverser(tab2['Name'].iloc[i],save_n)
    L=pd.DataFrame([[n0+i,save_n,'TRAIN',tab2['SEX'].iloc[i]]],columns=['ID','Name','ORIGIN','SEX'])
    tab2=tab2.append(L,ignore_index=True)
    

In [15]:
tab2

,ID,Name,ORIGIN,SEX
0,19,LibriSpeech/19-198-0000.flac,TRAIN,F
1,26,LibriSpeech/26-495-0000.flac,TRAIN,M
2,27,LibriSpeech/27-123349-0000.flac,TRAIN,M
3,32,LibriSpeech/32-21625-0000.flac,TRAIN,F
4,40,LibriSpeech/40-121026-0000.flac,TRAIN,F
...,...,...,...,...
799,9574,LibriSpeech/9574.flac,TRAIN,M
800,9575,LibriSpeech/9575.flac,TRAIN,M
801,9576,LibriSpeech/9576.flac,TRAIN,M
802,9577,LibriSpeech/9577.flac,TRAIN,M


In [16]:
tab=tab[tab['ORIGIN']=='TEST'].append(tab2)

In [17]:
tab

,ID,Name,ORIGIN,SEX
0,39,LibriSpeech/39-121914-0000.flac,TEST,F
1,229,LibriSpeech/229-130880-0000.flac,TEST,M
2,254,LibriSpeech/254-12312-0000.flac,TEST,M
3,322,LibriSpeech/322-124146-0000.flac,TEST,F
4,412,LibriSpeech/412-126975-0000.flac,TEST,M
...,...,...,...,...
799,9574,LibriSpeech/9574.flac,TRAIN,M
800,9575,LibriSpeech/9575.flac,TRAIN,M
801,9576,LibriSpeech/9576.flac,TRAIN,M
802,9577,LibriSpeech/9577.flac,TRAIN,M


In [18]:
tab.to_csv('SPEAKERS_TYPE_extended.csv',';',index=False)